In [4]:
%load_ext autoreload
%autoreload 2

import sys
import os
import dotenv
from pathlib import Path

env_file = "../.env"

if os.path.exists(env_file):
    dotenv.load_dotenv(env_file, verbose=True)
    print("Loaded environment variables from .env file.")

cwd = os.getcwd()
# for some reason appending to PATH you need it to be string
sys.path.append(str(Path(cwd).parent / "src"))
hf_access_token = os.getenv("HUGGINGFACE_API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loaded environment variables from .env file.


In [5]:
from research_tools.gpu import get_gpus_available

os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([str(i) for i in get_gpus_available()])
# os.environ["CUDA_VISIBLE_DEVICES"] = ",".join([2])

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
assert device == torch.device("cuda")

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "HuggingFaceH4/zephyr-7b-beta"
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, trust_remote_code=True, token=hf_access_token
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/mnt/align4_drive/data/huggingface/hub/models--HuggingFaceH4--zephyr-7b-beta/.no_exist/892b3d7a7b1cf10c7a701c60881cd93df615734c/adapter_config.json'


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
class CustomHook(torch.nn.Module):
    def __init__(self, module, hook_fn):
        super().__init__()
        self.module = module
        self.hook_fn = hook_fn
        self.enabled = True

    def forward(self, *args, **kwargs):
        if self.enabled:
            return self.hook_fn(self.module(*args, **kwargs))
        else:
            return self.module(*args, **kwargs)


def _remove_hook(parent, target):
    for name, module in parent.named_children():
        if name == target:
            setattr(parent, name, module.module)
            return


def insert_hook(parent, target, hook_fn):
    hook = None
    for name, module in parent.named_children():
        if name == target and hook is None:
            hook = CustomHook(module, hook_fn)
            setattr(parent, name, hook)
        elif name == target and hook is not None:
            _remove_hook(parent, target)
            raise ValueError(
                f"Multiple modules with name {target} found, removed hooks"
            )

    if hook is None:
        raise ValueError(f"No module with name {target} found")

    return hook

In [12]:
# add 1 to activations
def hook_fn(x):
    return x + 1


module = model.model.layers[7]
hook = insert_hook(module, "mlp", hook_fn)

In [13]:
def use_model(model, text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(
            device
        )
        model(**inputs)

In [14]:
text = "Oh say can you see, by the dawn's early light, What so proudly we hailed at the twilight's last gleaming? Whose broad stripes and bright stars through the perilous fight, O'er the ramparts we watched were so gallantly streaming? And the rocket's red glare, the bombs bursting in air, Gave proof through the night that our flag was still there; Oh, say does that star-spangled banner yet wave O'er the land of the free and the home of the brave?"

In [17]:
%timeit use_model(model, text)

28.4 ms ± 116 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
_remove_hook(module, "mlp")

In [ ]:
# now use register_forward_hook
def hook_fn(module, input, output):
    return output + 1


module = model.model.layers[7]
hook = module.mlp.register_forward_hook(hook_fn)

%timeit use_model(model, text)

28.4 ms ± 94.6 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


: 